<a href="https://colab.research.google.com/github/Flo909/GraspandLiftDetection/blob/main/Copy_of_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

import tensorflow

from matplotlib import pyplot as plt


# Training will be significantly faster if GPU is available. In Colab, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")
# set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


GPU is available


Import data:
Download the train.zip file from https://www.kaggle.com/c/grasp-and-lift-eeg-detection/data and
explore the dataset. This file contains the first 8 series for each subject. (We will be only using
train.zip for the project.)
There are two files for each subject + series combination:
● the *_data.csv files contain the raw 32 channels EEG data (sampling rate 500Hz)
● the *_events.csv files contains the ground truth frame-wise labels for all events

NOTE: to import the data you have to log into kaggle and create an API token. From there you upload the kaggle.json file that will be downloaded when creating the API token and upload it into the connected google drive account for colab. Then give colab permission to access the files when running the below lines.

In [7]:
! pip install kaggle

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [20]:
! kaggle competitions download grasp-and-lift-eeg-detection --force

100% 1.02G/1.02G [00:47<00:00, 24.2MB/s]
100% 1.02G/1.02G [00:47<00:00, 22.9MB/s]


In [4]:
import zipfile
import pandas as pd
import os

# Open the zip file
with zipfile.ZipFile('grasp-and-lift-eeg-detection.zip', 'r') as zip_ref:
    # Extract all files
    zip_ref.extractall('grasp-and-lift-eeg-detection')

# List the extracted files
extracted_files = zip_ref.namelist()

display(extracted_files)

['sample_submission.csv.zip', 'test.zip', 'train.zip']

In [6]:
# Function to read data and labels
def read_data_and_labels(zip_file):
    train_data = {}
    test_data = {}
    labels = {}

    # Extracted folder name will be the same as the zip file name without the extension
    extracted_folder = os.path.splitext(zip_file)[0]

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all files
        zip_ref.extractall(extracted_folder)

    extracted_files = zip_ref.namelist()

    for file_name in extracted_files:
        if file_name.endswith('_data.csv'):
            subject_id, series = file_name.split('_')[:2]
            print(series)
            df = pd.read_csv(os.path.join(extracted_folder, file_name))
            if series == 'series7' or series == 'series8':
                test_data.setdefault(subject_id, []).append(df)
            else:
                train_data.setdefault(subject_id, []).append(df)

        elif file_name.endswith('_events.csv'):
            subject_id = file_name.split('_')[0]
            labels_df = pd.read_csv(os.path.join(extracted_folder, file_name))
            labels[subject_id] = labels_df

    return train_data, test_data, labels

# Read data and labels
train_data, test_data, labels = read_data_and_labels('grasp-and-lift-eeg-detection/train.zip')



series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8
series1
series2
series3
series4
series5
series6
series7
series8


In [7]:
print(train_data.keys())
print(test_data.keys())

# Example: Accessing data for subject '1'
subject_id = 'train/subj2'
train_data_subject_1 = pd.concat(train_data[subject_id], ignore_index=True)
test_data_subject_1 = pd.concat(test_data[subject_id], ignore_index=True)
labels_subject_1 = labels[subject_id]

# Now you have train and test data along with labels for each subject

dict_keys(['train/subj10', 'train/subj11', 'train/subj12', 'train/subj1', 'train/subj2', 'train/subj3', 'train/subj4', 'train/subj5', 'train/subj6', 'train/subj7', 'train/subj8', 'train/subj9'])
dict_keys(['train/subj10', 'train/subj11', 'train/subj12', 'train/subj1', 'train/subj2', 'train/subj3', 'train/subj4', 'train/subj5', 'train/subj6', 'train/subj7', 'train/subj8', 'train/subj9'])
